<a href="https://colab.research.google.com/github/shibakyrc0123-arch/A-Gps-Versiones/blob/main/Ver_1_5_%7C_A_Gps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from datetime import timedelta
import time

# --- CONFIGURACIÓN ---
bodega_coords = [
    (4.101740989185771, -73.65614987790829),
    (4.10275779142336, -73.65779357282884),
    (4.103576674506101, -73.65722519691684),
    (4.103303969059117, -73.65525092470286)
]
bodega_polygon = Polygon(bodega_coords)
geolocator = Nominatim(user_agent="tracker_final_v20")

def obtener_direccion(lat, lon):
    try:
        # time.sleep(0.5) # Descomenta si recibes errores de timeout
        location = geolocator.reverse((lat, lon), exactly_one=True)
        # Tomamos solo la parte principal de la dirección
        return location.address.split(',')[0] if location else "Ubicación desconocida"
    except:
        return "Ubicación mapa"

def encontrar_archivo():
    rutas = ['/content/', '/content/sample_data/']
    for r in rutas:
        if os.path.exists(r):
            for f in os.listdir(r):
                if (f.endswith('.xls') or f.endswith('.xlsx')) and "california" not in f:
                    return os.path.join(r, f)
    return None

def limpiar_coordenada(coord_str):
    """
    Convierte strings de coordenadas a (lat, lon).
    Soporta: "4.123, -73.123" (con coma) y "4.123 -73.123" (con espacio).
    """
    s = str(coord_str).strip()
    if not s or s.lower() == 'nan': return None

    try:
        # Reemplazamos coma por espacio y dividimos por espacios
        # Esto unifica el formato a una lista de partes
        partes = s.replace(',', ' ').split()

        # Filtramos partes vacías
        partes = [p for p in partes if p.strip()]

        if len(partes) >= 2:
            return float(partes[0]), float(partes[1])
        return None
    except:
        return None

def generar_reporte_final():
    archivo = encontrar_archivo()
    if not archivo: return "❌ No encuentro el archivo."

    print(f"📂 Procesando: {archivo}")
    try:
        df = pd.read_excel(archivo)
    except:
        try:
            df = pd.read_csv(archivo)
        except:
            return "❌ Error leyendo formato archivo."

    # Normalizar columnas
    df.columns = df.columns.str.upper().str.strip()

    if 'FHEVENTO' not in df.columns: return "❌ Falta columna FHEVENTO"

    # Ordenar y filtrar
    df['FHEVENTO'] = pd.to_datetime(df['FHEVENTO'])
    df = df.sort_values(by='FHEVENTO').reset_index(drop=True)

    if not df.empty:
        anio = df['FHEVENTO'].iloc[0].year
        df = df[df['FHEVENTO'].dt.year == anio]

    reporte = []

    # Variables de Estado
    ya_reporto_encendido = False
    dentro_bodega = False

    estacionado_activo = False
    est_inicio = None
    est_fin = None
    est_coords = None

    print("⏳ Generando reporte narrativo...")

    for i, row in df.iterrows():
        raw_evento = str(row['EVENTO']).upper().strip()
        fecha = row['FHEVENTO']
        hora_fmt = fecha.strftime('%I:%M %p').lower().replace("am", "a.m.").replace("pm", "p.m.")

        # 1. COORDENADAS
        res = limpiar_coordenada(row['COORDS'])
        if res is None: continue
        lat, lon = res
        punto = Point(lat, lon)

        # --- LÓGICA BODEGA ---
        en_bodega_ahora = bodega_polygon.contains(punto)

        if en_bodega_ahora and not dentro_bodega:
            reporte.append(f"-{hora_fmt} Entrada Bodega Monte Carlo")
            dentro_bodega = True
        elif not en_bodega_ahora and dentro_bodega:
            reporte.append(f"-{hora_fmt} Salida Bodega Monte Carlo")
            dentro_bodega = False

        # --- LÓGICA ENCENDIDO ---
        if "ENCENDIDO" in raw_evento and not ya_reporto_encendido:
            lug = "Bodega Monte Carlo" if dentro_bodega else obtener_direccion(lat, lon)
            reporte.append(f"-{hora_fmt} Se enciende en {lug}")
            ya_reporto_encendido = True

        # --- LÓGICA ESTACIONADO ---
        es_quieto = False

        # Detectar quietud (por nombre o por cercanía al estacionamiento actual)
        if "ESTACIONADO" in raw_evento:
            es_quieto = True
        elif estacionado_activo:
            dist = geodesic(est_coords, (lat, lon)).meters
            if dist < 60: es_quieto = True

        if es_quieto:
            if not estacionado_activo:
                estacionado_activo = True
                est_inicio = fecha
                est_coords = (lat, lon)
            est_fin = fecha
        else:
            # Fin del estacionamiento
            if estacionado_activo:
                mins = (est_fin - est_inicio).total_seconds() / 60
                if mins > 2: # Mínimo 2 minutos
                    lug = "Bodega Monte Carlo" if dentro_bodega else obtener_direccion(*est_coords)
                    h_ini = est_inicio.strftime('%I:%M')
                    h_fin = est_fin.strftime('%I:%M %p').lower().replace("am", "a.m.").replace("pm", "p.m.")

                    reporte.append(f"-{h_ini} / {h_fin} Vehículo estacionado en {lug}")

                estacionado_activo = False
                est_inicio = None
                est_fin = None

        # --- LÓGICA CIUDAD ---
        if ("ENTRADA" in raw_evento or "SALIDA" in raw_evento) and "BODEGA" not in raw_evento:
             reporte.append(f"-{hora_fmt} {raw_evento.title()}")

    # Cierre final (si el archivo acaba y sigue estacionado)
    if estacionado_activo:
        mins = (est_fin - est_inicio).total_seconds() / 60
        if mins > 2:
            lug = "Bodega Monte Carlo" if dentro_bodega else obtener_direccion(*est_coords)
            h_ini = est_inicio.strftime('%I:%M')
            h_fin = est_fin.strftime('%I:%M %p').lower().replace("am", "a.m.").replace("pm", "p.m.")
            reporte.append(f"-{h_ini} / {h_fin} Vehículo estacionado en {lug}")

    print("✨ Completado.")
    return ", ".join(reporte) + "."

# --- EJECUCIÓN ---
try:
    resultado = generar_reporte_final()
    print("\n--- REPORTE FINAL ---")
    print(resultado)
except Exception as e:
    print(f"Error inesperado: {e}")

📂 Procesando: /content/sample_data/reporte-NCX43H-15-01-2026-10-06-12.xlsx
⏳ Generando reporte narrativo...
✨ Completado.

--- REPORTE FINAL ---
-07:58 a.m. Se enciende en Bistronomy, -11:30 / 08:04 a.m. Vehículo estacionado en Bistronomy, -02:41 / 02:46 p.m. Vehículo estacionado en Calle 35A, -05:46 / 08:07 a.m. Vehículo estacionado en Centro Comercial Primavera Urbana.
